In [1]:
using Revise

In [7]:
using ModelVerification

  ** incremental compilation may be fatally broken for this module **



ERROR: LoadError: 

UndefVarError: Model not defined
Stacktrace:
 [1] top-level scope
   @ ~/ModelVerification.jl/src/branching/search.jl:6
 [2] include(mod::Module, _path::String)
   @ Base ./Base.jl:419
 [3] include(x::String)
   @ ModelVerification ~/ModelVerification.jl/src/ModelVerification.jl:1
 [4] top-level scope
   @ ~/ModelVerification.jl/src/ModelVerification.jl:101
 [5] include
   @ ./Base.jl:419 [inlined]
 [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
   @ Base ./loading.jl:1554
 [7] top-level scope
   @ stdin:1
in expression starting at /home/verification/ModelVerification.jl/src/branching/search.jl:6
in expression starting at /home/verification/ModelVerification.jl/src/ModelVerification.jl:1
in expression starting at stdin:1


ErrorException: Failed to precompile ModelVerification [6d061d49-e389-4b69-b84f-d5b218a5bedb] to /home/verification/.julia/compiled/v1.8/ModelVerification/jl_Zugf9R.

In [3]:
using LazySets
using PyCall
using CSV
using ONNX
using Flux
using Test
using NNlib
using ONNXNaiveNASflux
using NaiveNASflux
# using DataFrames
# import Flux: flatten

In [4]:
# using Flux: onehotbatch, onecold, flatten
# using Flux.Losses: logitcrossentropy
# using Statistics: mean
using CUDA
using MLDatasets: CIFAR10, MNIST
using MLUtils: splitobs, DataLoader
using Accessors
using Profile
using LinearAlgebra

In [ ]:
function test_mlp(prop_method)
    small_nnet_file = "../test/networks/small_nnet.nnet"
    # small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
    small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
    flux_model = Flux.Chain(small_nnet)
    # println(flux_model)
    # println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)              [0,4]
    # println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5  [2.5, 18.5]
    # println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5                    [18.5, 114.5]
    in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected out: [18.5, 114.5]
    out_violated    = Hyperrectangle(low = [19], high = [114]) # 20.0 ≤ y ≤ 90.0
    out_holds = Hyperrectangle(low = [18], high = [115.0]) # -1.0 ≤ y ≤ 50.0
    comp_violated    = Complement(Hyperrectangle(low = [10], high = [19])) # y ≤ 10.0 or 19 ≤ y
    comp_holds    = Complement(Hyperrectangle(low = [115], high = [118])) # y ≤ 10.0 or 18 ≤ y
    info = nothing
    search_method = BFS(max_iter=100, batch_size=10)
    split_method = Bisect(1)
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_violated)).status == :violated
end
@timed begin
    for i in 1:100
        test_mlp(Ai2z())
        # test_mlp(Crown())
        # test_mlp(Ai2s())
    end
end

In [5]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(512, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
Flux_model = model
image_shape = (32, 32, 3, 5)
println(image_seeds)

UndefVarError: UndefVarError: BFS not defined

In [6]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_seeds, output_set))

UndefVarError: UndefVarError: ImageStar not defined

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_seeds, output_set))

In [ ]:
model = Chain([
    Flux.flatten,
    Dense(784, 200, relu),
    Dense(200, 10)
])
image_seeds = [MNIST(:train)[i].features for i in 1:1]
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/debug.onnx"
Flux_model = model
image_shape = (28, 28, 1, 1)

In [134]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    Conv((3, 3), 8 => 8, relu, pad=SamePad(), stride=(2, 2)),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(128, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
Flux_model = model
image_shape = (32, 32, 3, 2)

(32, 32, 3, 2)

In [ ]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [65]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

Set(Any["BatchNormalization", "Flatten", "Relu", "Conv", "Gemm", "AveragePool", "Add"])
Image Zono reach
0.0


(value = BasicResult(:holds), time = 0.009228519, bytes = 1409392, gctime = 0.0, gcstats = Base.GC_Diff(1409392, 12, 0, 22416, 26, 0, 0, 0, 0))

In [5]:
model = Chain([
    Conv((3, 3), 3 => 128, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(128),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 128 => 128, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    ConvTranspose((3, 3), 128 => 128, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(32768, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)

BallInf{Float64, Vector{Float64}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0)

In [6]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Conv((3, 3), 3 => 128, pad=1, stride=2)
│   summary(x) = 32×32×3×1 Array{Float64, 4}
└ @ Flux /home/tianhao/.julia/packages/Flux/EHgZm/src/layers/stateless.jl:60


3.5315663e10


(value = BasicResult(:unknown), time = 73.548692584, bytes = 18696768786, gctime = 1.790452061, gcstats = Base.GC_Diff(18696768786, 143, 10, 41435284, 16807, 340, 1790452061, 11, 2))

In [56]:
Profile.clear()
@profile verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

BasicResult(:holds)

In [45]:
open("./prof.txt", "w") do s
    Profile.print(IOContext(s, :displaysize => (24, 500)))
end